## Ecrire la description du problème sur moment de flemme

In [87]:
import torch
import numpy as np

In [88]:
def u_exact(x):
    return 2 + x

def grad_u():
    return 2

In [89]:
nb_points = 4
omega = torch.linspace(0, 1, nb_points) # boundary included
sol = u_exact(omega)

In [90]:
# MODEL DEFINITION #

class Block(torch.nn.Module):
    def __init__(self, m) -> None:
        super(Block, self).__init__()
        self.fc = torch.nn.Linear(in_features=m, out_features=m)
        # self.activation = torch.nn.Tanh()
        self.activation = lambda x: torch.maximum(x**3, torch.tensor(0.))

    def forward(self, x):
        identity = x
        x = self.fc(x)
        x = self.activation(x)
        x = self.fc(x)
        x = self.activation(x)
        return x + identity
    
class Model(torch.nn.Module):
    def __init__(self, input_dim, hidden_layer_dim, nb_blocks) -> None:
        super(Model, self).__init__()
        self.fc_in = torch.nn.Linear(in_features=input_dim, out_features=hidden_layer_dim)
        self.blocks = torch.nn.ModuleList([Block(hidden_layer_dim) for _ in range(nb_blocks)])
        self.fc_out = torch.nn.Linear(hidden_layer_dim, 1)

    def forward(self, x):
        x = self.fc_in(x)
        for block in self.blocks:
            x = block(x)
        x = self.fc_out(x)
        return x

In [95]:
num_epochs = 100
lr = 1e-4
beta_1 = 0.5
beta_2 = 0.999
gamma = 500
torch.manual_seed(0)

# MODEL INITIALIZATION #
gen = Model(input_dim=1, hidden_layer_dim=10, nb_blocks=1)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr, betas=(beta_1, beta_2))

for epoch in range(10):
    mc_omega = torch.rand(10) 
    mc_omega_unsqueeze = mc_omega.unsqueeze(1) # we add a dimension to match the input dimension of the model
    mc_omega_unsqueeze.requires_grad = True

    boundary_left = torch.zeros(1).unsqueeze(1)
    boundary_right = torch.ones(1).unsqueeze(1)

    output_mc_omega = gen(mc_omega_unsqueeze)
    output_mc_boundary_left = gen(boundary_left)
    output_mc_boundary_right = gen(boundary_right)
    # print(output_mc_boundary_right)

    grad_output_mc = torch.ones_like(output_mc_omega)
    grad_output_mc = torch.autograd.grad(output_mc_omega, mc_omega_unsqueeze, grad_output_mc, create_graph=True)[0]

    l1 = 0.5 * torch.mean(grad_output_mc.square())
    l2 = gen(boundary_left) - 2 + gen(boundary_right) - 3
    l = l1 - l2

    gen_opt.zero_grad()
    l.backward()
    gen_opt.step()

In [96]:
print(gen(omega.unsqueeze(1)))

tensor([[0.2818],
        [0.3324],
        [0.3824],
        [0.4305]], grad_fn=<AddmmBackward0>)
